# Message Passing Neural Networks

- Gated Graph Neural Networks의 일반화 버전이다.

- edge가 feature를 가질 수 있다.
    - 엣지의 feature(e_u,v)(message(m_v^k)도 노드, 엣지 피쳐를 사용해서 업데이트)
    - 그림 추후 첨부
- update node state by LSTM, GRU
    - h_v^k = U(h_v^(k-1), m_v^k)

# Creating Message Passing Networks

![mpn1](/img/mpn1.png)

Generalizing the convolution operator to irregular domains is typically expressed as a neighborhood aggregation or message passing scheme. With x(k−1)i∈RF denoting node features of node i in layer (k−1) and ei,j∈RD denoting (optional) edge features from node i to node j, message passing graph neural networks can be described as
x(k)i=γ(k)(x(k−1)i,□j∈N(i)ϕ(k)(x(k−1)i,x(k−1)j,ei,j)),

- conv operator를 irregular domain으로 일반화하는 것은 주로 neighborhood aggregation이나 message passing으로 나타난다. 

- x_i^(k-1)은 (k-1) layer의 i번째 노드를 나타내고, e_i,j는 edge의 특성을 나타낸다.

- message passing graph neural network는 위 식 처럼 나타내어진다.

where □ denotes a differentiable, permutation invariant function, e.g., sum, mean or max, and γ and ϕ denote differentiable functions such as MLPs (Multi Layer Perceptrons).

- i와 연결되어있는 모든 노드와 이 계산을 진행한다. 

# The "Message Passing" Base Class

PyTorch Geometric provides the ```torch_geometric.nn.MessagePassing``` base class, which helps in creating such kinds of message passing graph neural networks by automatically taking care of message propagation. The user only has to define the functions ϕ , i.e. message(), and γ , .i.e. update(), as well as the aggregation scheme to use, .i.e. aggr='add', aggr='mean' or aggr='max'.

- 유저는 오직 ϕ(message())(MLP)와 γ(update())만 구현하면된다. 
- 더해서 ```aggregation scheme```까지.. (add, mean, max)

```torch_geometric.nn.MessagePassing(aggr="add", flow="source_to_target")```: Defines the aggregation scheme to use ("add", "mean" or "max") and the flow direction of message passing (either "source_to_target" or "target_to_source").

- aggregation은 message passing과 거의 같은말

```torch_geometric.nn.MessagePassing.propagate(edge_index, size=None, dim=0, **kwargs)```
The initial call to start propagating messages. Takes in the edge indices and all additional data which is needed to construct messages and to update node embeddings. 
Note that propagate() is not limited to exchange messages in symmetric adjacency matrices of shape [N, N] only, but can also exchange messages in general sparse assignment matrices, .e.g., bipartite graphs, of shape [N, M] by passing size=(N, M) as an additional argument. If set to None, the assignment matrix is assumed to be symmetric. For bipartite graphs with two independent sets of nodes and indices, and each set holding its own information, this split can be marked by passing the information as a tuple, e.g. x=(x_N, x_M). Furthermore, the dim attribute indicates along which axis to propagate.

- 메세지를 propagate하는 데 부르는 맨 처음 call
- edge indice와 message를 만드는데 드는 추가 데이터를 받아서 노드 임베딩을 업데이트
- size가 None으로 설정 되어있다면, symmetric으로 간주. 물론 대칭이 아니어도 상관 없음.

```torch_geometric.nn.MessagePassing.message()```: '
Constructs messages to node i in analogy to ϕ for each edge in (j,i)∈E if flow="source_to_target" and (i,j)∈E if flow="target_to_source". 
Can take any argument which was initially passed to propagate(). 
In addition, tensors passed to propagate() can be mapped to the respective nodes i and j by appending _i or _j to the variable name, .e.g. x_i and x_j.

- 만약 flow가 "source to target"이라면, (j,i)의 each edge에 대한 i번째 노드로의 message 작성

```torch_geometric.nn.MessagePassing.update()```: Updates node embeddings in analogy to γ for each node i∈V. Takes in the output of aggregation as first argument and any argument which was initially passed to propagate().

- 노드 임베딩을 x(k)i=γ(k)(x(k−1)i,□j∈N(i)ϕ(k)(x(k−1)i,x(k−1)j,ei,j)) 에 맞추어 update